
* Summary
```
1) 다른 데이터에 적용한다면 층 구성을 어떻게 할것인가? 입력-은닉-출력층으로 구성됩니다.
2) 예측하고자 하는 값이 분류(이진, 멀티클래스), 회귀인지에 따라 출력층 구성, loss 설정이 달라집니다. 
이건 예제를 몇 개 연습해 보면서 익히는 것을 추천해요.
3) 분류, 회귀에 따라 측정 지표 정하기
4) 활성화함수는  relu 를 사용, optimizer 로는 adam 을 사용하면  baseline 정도의 스코어가 나옵니다.
5) fit 을 할 때 epoch 를 통해 여러 번 학습을 진행하는데 이 때, epoch 수가 많을 수록 대체적으로 좋은 성능을 내지만 과대적합(오버피팅)이 될 수도 있습니다.
6) epoch 수가 너무 적다면 과소적합(언더피팅)이 될 수도 있습니다. 
```

## 텐서플로 분류 모델 만들기
* [첫 번째 신경망 훈련하기: 기초적인 분류 문제  |  TensorFlow Core](https://www.tensorflow.org/tutorials/keras/classification)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("https://bit.ly/telco-csv", index_col="customerID")
df.shape

## Preprocessing

In [ ]:
# TotalCharges 가 수치 타입이 아니기 때문에 수치 연산을 위해 숫자 형태로 변경합니다.
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

In [ ]:
label_name = "Churn"

In [ ]:
df = df.dropna()
df.shape

In [ ]:
X_raw = df.drop(columns=label_name)
y = (df[label_name] == "Yes").astype("float16")

## One-Hot-Encoding

In [ ]:
X = pd.get_dummies(X_raw)
X.head(2)

## train test split

In [ ]:
# 학습세트와 예측세트를 분리합니다.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)

In [ ]:
input_shape = X_train.shape[1]
input_shape

## model

In [ ]:
import tensorflow as tf
seed = 42
tf.random.set_seed(seed)

model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(input_shape)),
  tf.keras.layers.Dense(256, activation='swish'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='swish'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid') 
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
X_train.dtypes.describe()

## fit

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

history = model.fit(X_train, y_train, 
          epochs=100,
          use_multiprocessing=True,
          callbacks=[early_stop],
          verbose=0)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

## predict

In [ ]:
y_pred = model.predict(X_test)
y_pred[0]

## evaluate

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\n테스트 정확도:', test_acc)

In [ ]:
y_predict = (y_pred.flatten() > 0.5).astype(int)
y_predict

In [ ]:
(y_test == y_predict).mean()